In [34]:
# Importing the libraries
from google.cloud import bigquery
import numpy as np
import pandas as pd

In [35]:
# Authenticating the user
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

Authenticated


In [36]:
# Creting a client object
project_id = "sqlproject-302612"
client = bigquery.Client(project=project_id)

In [37]:
# Fetchiing the data
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))


In [38]:
# Listing the name of the tables
for table in tables:
  print(table.table_id)

commits
contents
files
languages
licenses
sample_commits
sample_contents
sample_files
sample_repos


In [41]:
# Question: How many files are covered by each type of software license?
table_licenses_ref = dataset_ref.table("licenses")
table_licenses = client.get_table(table_licenses_ref)
client.list_rows(table_licenses, max_results=5).to_dataframe()


,repo_name,license
0,tmtowtdi/perl6_exercises,artistic-2.0
1,jonathanstowe/Pg-Notify,artistic-2.0
2,tukhoi/PodCricket,artistic-2.0
3,gingeralesy/noise-map-generator,artistic-2.0
4,Cousk1927/Google,artistic-2.0


In [42]:
table_sample_files_ref = dataset_ref.table("sample_files")
table_sample_files = client.get_table(table_sample_files_ref)
client.list_rows(table_sample_files, max_results=5).to_dataframe()

,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


In [50]:
query = ''' SELECT L.license, COUNT(1) AS number_of_files
            FROM `bigquery-public-data.github_repos.sample_files` AS sf
            INNER JOIN `bigquery-public-data.github_repos.licenses` AS L
                ON sf.repo_name = L.repo_name
            GROUP BY L.license
            ORDER BY number_of_files DESC
        '''
      

In [51]:
# Submitting the query
query_job = client.query(query)
n_file = query_job.to_dataframe()
n_file

,license,number_of_files
0,mit,20418149
1,gpl-2.0,16793004
2,apache-2.0,7106727
3,gpl-3.0,4902192
4,bsd-3-clause,2923920
5,agpl-3.0,1292022
6,lgpl-2.1,792881
7,bsd-2-clause,694532
8,lgpl-3.0,563605
9,mpl-2.0,472843
